# Домашнее задание №1

### 1. Написать на PyTorch глубокую сеть. Проверить форвард пасса.

In [2]:
# Загружаем библиотеки
import pandas as pd
import numpy as np
import torch
from torch import nn

In [3]:
class MyModel(nn.Module):
    def __init__(self,):
        super().__init__()

        self.layer1 = nn.Linear(1, 100)
        self.activ = nn.ReLU()
        self.layer2 = nn.Linear(100, 1)
        self.do = nn.Dropout(0.1)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.activ(x)
        x = self.do(x)
        x = self.layer2(x)
                        
        return x

In [4]:
x = (torch.rand(1000)-0.4)*3
y = x**3 + torch.randn(1)*0.2

In [5]:
model = MyModel()
model.train() 
optim = torch.optim.Adam(model.parameters())
loss_func = nn.MSELoss()

In [6]:
n_epochs = 10
for epoch in range(n_epochs):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        predict = model(val.unsqueeze(dim=0))
        loss = loss_func(predict, t)
        loss.backward()
        optim.step()
    print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

C:\Users\150ho\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0, step: 999, loss: 0.16550229489803314
epoch: 1, step: 999, loss: 0.12685929238796234
epoch: 2, step: 999, loss: 0.061052531003952026
epoch: 3, step: 999, loss: 0.0025708989705890417
epoch: 4, step: 999, loss: 0.00047343346523121
epoch: 5, step: 999, loss: 0.004774009808897972
epoch: 6, step: 999, loss: 0.033541977405548096
epoch: 7, step: 999, loss: 0.0007575800991617143
epoch: 8, step: 999, loss: 0.009023685939610004
epoch: 9, step: 999, loss: 0.011115718632936478


### 2. Написать адаптивный оптимизатор.

In [7]:
# Task 2
# Realize SGD Momentum optimizer
# velocity = momentum * velocity - lr * gradient
# w = w + velocity

In [8]:
# Класс линейный слой с леккции №1
class LinearLayer:
    def __init__(self, n_inp, n_out, activation='sigmoid'):
        self.w = np.random.randn(n_out, n_inp) * 0.1
        self.b = np.random.randn(n_out, 1) * 0.1
        if activation == 'sigmoid':
            self.activ = sigmoid
        if activation == 'relu':
            self.activ = relu
        elif activation == 'None':
            self.activ = None
        else:
            raise Exception(f'Unknown activation "{activation}"')
        self._clear_state()

    def _clear_state(self):
        self.lin = None
        self.inp = None
        self.d_w = None
        self.d_b = None

    def forward(self, x):
        self.inp = x
        self.lin = np.dot(self.w, x) + self.b
        activ = self.activ(self.lin) if self.activ is not None else self.lin

        return activ

    def backward(self, grad): # grad = d L / d z    Dout 
        # grad * dz / d lin
        if self.activ == sigmoid:
            grad_lin = sigmoid_backward(grad, self.lin) 
        elif self.activ == relu:
            grad_lin = relu_backward(grad, self.lin)
        else:
            grad_lin = grad
        # grad_lin * d lin / d w 
        m = self.inp.shape[1]
        self.d_w = np.dot(grad_lin, self.inp.T) / m    # d_in dOut
        # grad_lin * d lin / d b 
        self.d_b = np.sum(grad_lin, axis=1, keepdims=True) / m

        grad = np.dot(self.w.T, grad_lin)

        return grad

In [9]:
#для одного слоя (оптимицатор с лекции №1)
class SGDMomentum1:
    def __init__(self, model: LinearLayer, lr=0.001, momentum=0.99):
        self.lr = lr
        self.m = momentum
        self.model = model

        self.vel_w = np.zeros_like(model.w)
        self.vel_b = np.zeros_like(model.b)

    def step(self):
        self.vel_w = self.m * self.vel_w - self.lr * self.model.d_w
        self.vel_b = self.m * self.vel_b - self.lr * self.model.d_b

        self.model.w += self.vel_w
        self.model.b += self.vel_b

    def zero_grad(self):
        self.model.d_w = np.zeros_like(self.model.d_w)
        self.model.d_b = np.zeros_like(self.model.d_b)




#### Оптимизатор Адам

https://pythonguides.com/adam-optimizer-pytorch/

https://habr.com/ru/company/skillfactory/blog/525214/

https://towardsdatascience.com/how-to-implement-an-adam-optimizer-from-scratch-76e7b217f1cc

In [10]:
class AdamOptim():
    def __init__(self, eta=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.m_dw, self.v_dw = 0, 0
        self.m_db, self.v_db = 0, 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.eta = eta
    def update(self, t, w, b, dw, db):
        ## dw, db из текущей минибатча
        ## momentum beta 1
        # *** веса *** #
        self.m_dw = self.beta1*self.m_dw + (1-self.beta1)*dw
        # *** смещения *** #
        self.m_db = self.beta1*self.m_db + (1-self.beta1)*db

        ## rms beta 2
        # *** веса *** #
        self.v_dw = self.beta2*self.v_dw + (1-self.beta2)*(dw**2)
        # *** смещения *** #
        self.v_db = self.beta2*self.v_db + (1-self.beta2)*(db)

        ## коррекция смещения
        m_dw_corr = self.m_dw/(1-self.beta1**t)
        m_db_corr = self.m_db/(1-self.beta1**t)
        v_dw_corr = self.v_dw/(1-self.beta2**t)
        v_db_corr = self.v_db/(1-self.beta2**t)

        ## обновить веса и смещения
        w = w - self.eta*(m_dw_corr/(np.sqrt(v_dw_corr)+self.epsilon))
        b = b - self.eta*(m_db_corr/(np.sqrt(v_db_corr)+self.epsilon))
        return w, b

### 3. Решить задачу нахождения корней квадратного уравнения методом градиентного спуска.

In [11]:
# Task 3
# Find the roots of square equation by gradient descent
# x ** 2 - 6 * x + 4 = 0

# посчитать производную от преобразованной функции
# надо начать движение от начальной точки в направлении антиградиента с заданным шагом
# x = x - lr * grad(x)

https://ru.wikipedia.org/wiki/Дискриминант

In [12]:
# Найдем корни аналитически (через дискриминант)
# x ** 2 - 6 * x + 4 = 0
a = 1
b = -6
c = 4
D = b**2 - 4*a*c
x1 = (-b + D**0.5)/(2*a)
x2 = (-b - D**0.5)/(2*a)
print(x1, x2)

5.23606797749979 0.7639320225002102


https://www.programmersought.com/article/16521452441/

https://www.dmitrymakarov.ru/opt/gradient-02/

https://medium.com/analytics-vidhya/if-we-have-any-function-and-we-want-find-the-extremum-of-that-function-whether-it-is-minima-or-3dd53d89ea40

In [13]:
# заданная функция
def f1(x):
    return x**2 - 6*x + 4    

# производная функции
def grad1(x):
    return 2 * x - 6

In [14]:
# метод градиентного спуска 
def gradient_descent(x, lr, acc):
    x1 = x
    x2 = x1 - lr * grad1(x1)
    count = 1
    while f1(x2) > acc:
        x1 = x2
        x2 = x1 - lr * grad1(x1)
        count += 1
    return x2, count
    
gradient_descent(-10, 0.00001, 0.00001), gradient_descent(10, 0.00001, 0.00001)     

((0.763948124793057, 88011), (5.236067487546305, 57059))

In [15]:
# функция в квадрате
def f2(x):
    return (x**2 - 6*x + 4)**2     # x**4 - 12 * x**3 + 44*x**2 - 48 * x + 16

# производная функции в квадрате
def grad2(x):
    return 2 * (x ** 2 - 6 * x + 4) * (2 * x - 6)  # 4 * x**3 - 36 * x**2 + 88 * x - 48

In [16]:
# метод Ньютона
def newton(x, lr, acc):
    x1 = x
    x2 = x1 - lr * grad2(x1)
    count = 1
    while f2(x2) > acc:
        x1 = x2
        x2 = x1 - lr * grad2(x1)
        count += 1
    return x2, count

newton(-10, 0.001, 0.00001), newton(10, 0.001, 0.00001)

((0.7632443126002381, 174), (5.23677368251635, 176))

In [17]:
# Выводы:
# всегда ли сойдемся за приемлемое количество шагов? - Количество шагов зависит от LR и заданной точности (acc)
# важна ли начальная точка? - От начальной точки зависит в какой минимум будет производиться спуск
# как найти второй корень? - Выбрать другую начальную точку (шагать с дыух шагов)
# как вляет ЛР? - ЛР влияет на скорость спуска (количество шагов)